# Проба на основе "Случайный лес".

In [17]:
from pathlib import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
from sklearn.model_selection import train_test_split
import scipy as sp
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import metrics
from matplotlib.legend_handler import HandlerLine2D
from sklearn.tree import export_graphviz
import graphviz

## Анализ данных

Загрузим данные и посмотрим на них

In [18]:
df = pd.read_csv(
        Path('c:\data_quote\data_prepare_RTS_range_mvc_tpsl_sec\SPFB.RTS_00_range250_splice_2022.txt'),
        delimiter=','
    )
print("Количество наблюдений: ", df.shape[0])
print("Количество атрибутов: ", df.shape[1])

# Убираем ненужные колонки
df.drop(labels=['<DATE>', '<VOL>', '<MAX_VOLUME_PRICE>'], axis=1, inplace=True)
df.rename(columns={'<MAX_VOLUME_CLUSTER>': '<MAX_VOL>'}, inplace=True)  # Переименование колонки

pd.set_option('display.max_columns', None)

df

Количество наблюдений:  12778
Количество атрибутов:  12


,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<MAX_VOL>,<TP_SL>,<SEC>,<PER>
0,70000,159950.0,160200.0,159950.0,160200.0,135,1,1,0.08
1,70001,160600.0,160960.0,160710.0,160710.0,121,3,1,0.00
2,70002,160700.0,160730.0,160480.0,160480.0,147,3,4,0.52
3,70006,160470.0,160600.0,160350.0,160350.0,350,3,63,0.36
4,70109,160340.0,160510.0,160260.0,160260.0,212,3,275,0.56
...,...,...,...,...,...,...,...,...,...
12773,232158,144950.0,145170.0,144920.0,145170.0,263,0,638,0.52
12774,233236,145180.0,145370.0,145120.0,145120.0,227,0,317,0.16
12775,233753,145110.0,145200.0,144950.0,144950.0,247,-1,433,0.40
12776,234506,144940.0,145180.0,144930.0,145180.0,89,0,135,0.60


In [19]:
# Дальнейшая очистка данных
df = df.loc[df['<TP_SL>'] != 0]  # Убираем строки где <TP_SL> равен 0
df = df.loc[df['<HIGH>'] > df['<MAX_VOL>']]
df = df.loc[df['<MAX_VOL>'] > df['<LOW>']]
df

,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<MAX_VOL>,<TP_SL>,<SEC>,<PER>
